## X4, X5

##### X4: The percentage of subfunds holding the stock that are good, as defined by industry cluster.

##### X5: The mean portfolio weight of the stock for all good subfunds currently holding the stock, as defined by industry cluster.

Good funds: are subfunds that
 - 1) have top 3 correlation w/ their ETF within their parent fund
 - 2) a positive return over their ETF
 
Subfunds: defined by `industry cluster` or `size/value cluster`, a fund can be divided into at most 10 subfunds which represents all hodings in a specific sector. The benchmark to determine sector of each stock is the sector of the ETF of the highest correlation with the stock.

In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import *

In [2]:
# form13f_FF4 = pd.read_csv('data/form13f_FF4.csv', sep = '\t')
fund_prices_FF = pd.read_csv('data/fund_prices_FF.csv', sep = '\t')
good_subfund_labels = pd.read_csv('data/good_subfund_labels.csv')

In [3]:
good_subfund_labels["iPERIOD_END"] = pd.to_datetime(good_subfund_labels["iPERIOD_END"]).dt.date
good_subfund_labels["iFILING_DATE"] = pd.to_datetime(good_subfund_labels["iFILING_DATE"]).dt.date
fund_prices_FF["pSP_DATE"]= pd.to_datetime(fund_prices_FF["pSP_DATE"]).dt.date

In [4]:
good_subfund_labels
# 每个基金公司每6.30，9.30，12.31，3.31的股票hodings按时间序列排下来！
# 一个基金公司所有拉完再换下一个公司

,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iMARKET_VALUE,sector,value_size,filing_3mos,market_V_3mos,subfund_sector,...,value_size_close,value_s_close_3mos,return_x,sector_return,corr_sector,good_subfund_sector,return_y,value_size_return,corr_vs,good_subfund_vs
0,7195,219350105,2013-06-30,2013-08-12,4302000.0,Materials,Large-Cap Value,2013-05-01,4.079925e+06,7195_Materials,...,80.6506,74.4794,0.056152,0.071721,NaN,0,0.046351,0.082858,NaN,0
1,7195,478160104,2013-06-30,2013-08-12,5000000.0,Industrial,Large-Cap Value,2013-05-01,4.577717e+06,7195_Industrial,...,80.6506,74.4794,0.120541,0.119111,NaN,0,0.046351,0.082858,NaN,0
2,7195,369550108,2013-06-30,2013-08-12,4813000.0,Industrial,Large-Cap Value,2013-05-01,4.140179e+06,7195_Industrial,...,80.6506,74.4794,0.120541,0.119111,NaN,0,0.046351,0.082858,NaN,0
3,7195,902973304,2013-06-30,2013-08-12,4846000.0,Financial,Large-Cap Value,2013-05-01,4.278721e+06,7195_Financial,...,80.6506,74.4794,0.157916,0.104484,NaN,0,0.046351,0.082858,NaN,0
4,7195,494368103,2013-06-30,2013-08-12,4783000.0,Consumer Staples,Large-Cap Value,2013-05-01,5.025800e+06,7195_Consumer Staples,...,80.6506,74.4794,-0.014879,0.020061,NaN,0,0.046351,0.082858,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226142,1653866,46625H100,2018-06-30,2018-07-17,2098000.0,Financial,Large-Cap Growth,2018-05-01,2.065343e+06,1653866_Financial,...,180.2700,162.7800,-0.019259,0.005428,0.990428,0,0.085033,0.107446,0.842806,0
226143,1653866,654106103,2018-06-30,2018-07-17,2026000.0,Consumer Discretionary,Large-Cap Growth,2018-05-01,1.780955e+06,1653866_Consumer Discretionary,...,180.2700,162.7800,0.099774,0.090882,0.969240,1,0.085033,0.107446,0.842806,0
226144,1653866,22160K105,2018-06-30,2018-07-17,429000.0,Consumer Discretionary,Large-Cap Growth,2018-05-01,3.921858e+05,1653866_Consumer Discretionary,...,180.2700,162.7800,0.099774,0.090882,0.969240,1,0.085033,0.107446,0.842806,0
226145,1653866,464287119,2018-06-30,2018-07-17,559000.0,Consumer Discretionary,Large-Cap Growth,2018-05-01,5.109503e+05,1653866_Consumer Discretionary,...,180.2700,162.7800,0.099774,0.090882,0.969240,1,0.085033,0.107446,0.842806,0


In [6]:
good_subfund_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226147 entries, 0 to 226146
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   iCIK                 226147 non-null  int64  
 1   iCUSIP               226147 non-null  object 
 2   iPERIOD_END          226147 non-null  object 
 3   iFILING_DATE         226147 non-null  object 
 4   iMARKET_VALUE        226147 non-null  float64
 5   sector               226147 non-null  object 
 6   value_size           226147 non-null  object 
 7   filing_3mos          226147 non-null  object 
 8   market_V_3mos        226147 non-null  float64
 9   subfund_sector       226147 non-null  object 
 10  subfund_vs           226147 non-null  object 
 11  sector_close         226147 non-null  float64
 12  sector_close_3mos    226147 non-null  float64
 13  value_size_close     226147 non-null  float64
 14  value_s_close_3mos   226147 non-null  float64
 15  return_x         

In [5]:
fund_prices_FF
# 每天的所有股票，每天的都列完了才换下一天

,pSP_CUSIP,pSP_EXCHANGE,pSP_TICKER,pSP_DATE,pSP_VOLUME,pSP_OPEN,pSP_HIGH,pSP_LOW,pSP_CLOSE
0,464287150,N,ISI,2013-01-01,0,7.645,7.645,7.645,7.6450
1,78464A847,N,EMM,2013-01-01,0,0.990,0.990,0.990,0.9870
2,78464A854,N,ELR,2013-01-01,0,31.270,31.270,31.270,31.2681
3,74347R479,N,UVT,2013-01-01,0,0.940,0.940,0.940,0.9400
4,78464A805,N,TMW,2013-01-01,0,4.324,4.324,4.324,4.3240
...,...,...,...,...,...,...,...,...,...
5710060,437872104,O,HTBI,2018-08-14,20464,29.800,29.850,29.200,29.4500
5710061,00847J105,O,AGYS,2018-08-14,87154,15.850,16.080,15.750,15.9500
5710062,78425V104,O,SIFI,2018-08-14,9596,13.600,13.850,13.600,13.7500
5710063,253651103,N,DBD,2018-08-14,7108229,4.750,5.250,4.650,5.0000


## X4

In [11]:
x4 = []
for i in good_subfund_labels.iCUSIP.unique():
    stock_i = good_subfund_labels[good_subfund_labels['iCUSIP'] == i]
    num_good_subfund = len(stock_i[stock_i['good_subfund_sector'] == 1])
    num_subfund = len(stock_i)
    per_good_subfund = num_good_subfund / num_subfund
    x4.append(per_good_subfund)
df_X4 = pd.DataFrame({"pSP_CUSIP": good_subfund_labels.iCUSIP.unique(), "x4": x4})
df_X4

,pSP_CUSIP,x4
0,219350105,0.103015
1,478160104,0.129565
2,369550108,0.111588
3,902973304,0.263514
4,494368103,0.118902
...,...,...
4412,06740L444,0.000000
4413,921946794,0.500000
4414,464287119,0.666667
4415,46134L105,0.000000


## X5

In [9]:
x5 = []
for i in good_subfund_labels.iCUSIP.unique():
    stock_i_InGood = good_subfund_labels[(good_subfund_labels['iCUSIP'] == i) 
                                         & (good_subfund_labels['good_subfund_sector'] == 1)]
    portfolio_weights = []
    for index, row in stock_i_InGood.iterrows():
        all_good_fund = good_subfund_labels[(good_subfund_labels['iPERIOD_END'] == row['iPERIOD_END'])
                                           & (good_subfund_labels['subfund_sector'] == row['subfund_sector'])]
        
        all_market_value = all_good_fund.sum().iMARKET_VALUE
        stock_market_value = row['iMARKET_VALUE']
        portfolio_weight = stock_market_value / all_market_value
        portfolio_weights.append(portfolio_weight)
    stock_i_portfolio_weights = np.array(portfolio_weights).mean()
    x5.append(stock_i_portfolio_weights)
    
df_X5 = pd.DataFrame({"pSP_CUSIP": all_stocks, "x5": x5})

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

...

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_por

<ipython-input-9-555f77c54f6f>:14: RuntimeWarning: Mean of empty slice.
  stock_i_portfolio_weights = np.array(portfolio_weights).mean()
C:\Users\jingd\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'all_stocks' is not defined

In [10]:
df_X5 = pd.DataFrame({"pSP_CUSIP": good_subfund_labels.iCUSIP.unique(), "x5": x5})
df_X5

,pSP_CUSIP,x5
0,219350105,0.103309
1,478160104,0.174177
2,369550108,0.079706
3,902973304,0.155510
4,494368103,0.115197
...,...,...
4412,06740L444,NaN
4413,921946794,0.002946
4414,464287119,0.071317
4415,46134L105,NaN


In [ ]:
df_X4_5 = pd.merge(df_X4, df_X5, on = "pSP_CUSIP")
df_X4_5

In [16]:
example_df = form13f_FF4[(form13f_FF4['iCIK'] == 7195) & (form13f_FF4['iPERIOD_END'] == datetime.date(2013,6,30))]
example_df.sort_values(by=["sector"], inplace=True)
example_df

<ipython-input-16-8ffa29c05481>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  example_df.sort_values(by=["sector"], inplace=True)


,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
50,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205,2013-05-01,22.0784,3.872379e+06,4243000,0
28,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004,2013-05-01,76.9900,4.438103e+06,4705000,0
30,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946,2013-05-01,52.4000,5.002538e+06,4740000,0
35,39276293,7195,171232101,2013-06-30,2013-08-12,53980.0,4569000.0,0.019876,43 days,Consumer Discretionary,Large-Cap Growth,85.5200,53426.099158,2013-05-01,87.3000,4.664098e+06,4569000,0
49,39276304,7195,38259P508,2013-06-30,2013-08-12,5809.0,5114000.0,0.022247,43 days,Consumer Discretionary,Small-Cap Blend,447.3110,11432.761546,2013-05-01,414.4360,4.738148e+06,5114000,0
13,39276290,7195,053015103,2013-06-30,2013-08-12,71320.0,4911000.0,0.021364,43 days,Consumer Discretionary,Large-Cap Growth,63.9936,76742.049205,2013-05-01,58.7369,4.507590e+06,4911000,0
14,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414,2013-05-01,72.7600,4.559443e+06,4963000,0
46,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718,2013-05-01,108.2200,4.267831e+06,4564000,0
11,39276314,7195,494368103,2013-06-30,2013-08-12,49241.0,4783000.0,0.020807,43 days,Consumer Staples,Large-Cap Value,94.1450,50804.609910,2013-05-01,98.9241,5.025800e+06,4783000,0
24,39276320,7195,713448108,2013-06-30,2013-08-12,60983.0,4988000.0,0.021699,43 days,Consumer Staples,Large-Cap Value,83.8100,59515.570934,2013-05-01,82.2100,4.892775e+06,4988000,0
